In [1]:
# A resource is wood, brick, stone, glass, paper
#     [wood, brick, stone, glass, paper]
# Card is the basis for the game cards are given and taken away from players, 
# players never GAIN the abilities of the cards, they just gain or lose cards
import collections, functools, operator 


# Player Class

In [2]:
class Player:
    def __init__(self, name):
        self.name=name
        self.card_dict={}
        self.coins=7
    def get_name(self):
        return self.name
    
    def get_coin(self):
        return self.coins
    
    def put_card(self, card, card_cost):
        print(self.get_name() + " acquired " + card.get_name())
        self.card_dict[card.name]={"Attack": card.attack, "VP": card.vp,
                                   "Resource": card.resource, "Card Type": card.card_type}
        self.coins-=card_cost
        if card.get_coin_cost() > 0:
            print(self.get_name() + " spent {0} Coin(s)".format(card_cost))
    
    def get_resource_amount(self, resource):
        return dict(self.count_type("Resource").items()).get(resource)
        
    def count_type(self, action_type):
        t_sum=0
#         Deal with int types 
        if action_type in ["Attack", "VP"]: 
            for i in self.card_dict.values():
                t_sum +=i[action_type]
            return t_sum
#         Deal with tuple types (Resource)
        if action_type=="Resource":
            all=[]
            all.append({"Coin":self.coins})
            for i in self.card_dict.values():
                all.append(i[action_type])
            result = dict(functools.reduce(operator.add, 
                                           map(collections.Counter, all)))
            return result
        
    def discard_card(self, card):
#         This function handles discarding card to gain coins based off of number of Treasure Cards
        self.count_type("Treasure")
        
    def trade_ratio_calc(self, opp, m_dict): #return a dictionary with resource cost per resource missing EXCLUDES COIN COSTS 
        def trade_ratio_calc_child(self, opp, m_dict, ret):
            if len(m_dict) > 0:
                resource = next(iter(m_dict)) 
                p_resource_count = self.get_resource_amount(resource) if self.get_resource_amount(resource) is not None else 0
                o_resource_count = opp.get_resource_amount(resource) if opp.get_resource_amount(resource) is not None else 0
                #base cost is 2 + number opponent has do I need player dict? need player dict to get special resource and conversion cards
                ret[resource] = o_resource_count + 2 
                del m_dict[next(iter(m_dict))]
                trade_ratio_calc_child(self, opp, m_dict, ret)
            return ret
        if "Coin" in m_dict:
            del m_dict["Coin"]
        ret = {resource:0 for resource in m_dict.keys()} #initialize ret
        return trade_ratio_calc_child(self, opp, m_dict, ret)
    
    def add_card(self, card, opp):
        flg=1
        m_dict = {}
        #determine if you can afford card. 
        for c_resource, c_amount in zip(card.get_cost().keys(), card.get_cost().values()):
            if self.get_resource_amount(c_resource) is None or self.get_resource_amount(c_resource) < c_amount:
                #avoid contrasting None vs int type 
                if self.get_resource_amount(c_resource) == None:
                    m_dict[c_resource] = c_amount
                else:
                    m_dict[c_resource] = c_amount - self.get_resource_amount(c_resource)
                flg=0
                #set flag here to jump into if else below 
                
        if flg: # add card to player and remove necessary coins
            self.put_card(card,card.get_coin_cost())            
        else: #check for 
            #create copy of m_dict to multiply trade_ratio by m_dict to find total cost after adjusting for BaseSpecial
            m_dict_copy = m_dict.copy()
            trade_ratios = self.trade_ratio_calc(opp, m_dict_copy)
            #basespecial
            if self.get_resource_amount("BaseSpecial"):
                keys_to_extract = ["Wood", "Stone", "Brick"]
                base_trade_ratio = {}
                for key in keys_to_extract:
                    if key in trade_ratios:
                        base_trade_ratio[key] = trade_ratios[key]
                for i in range(self.get_resource_amount("BaseSpecial")):
                    if m_dict[max(base_trade_ratio, key=base_trade_ratio.get)]>0:
                        m_dict[max(base_trade_ratio, key=base_trade_ratio.get)] -= 1
            #AdvancedSpecial
            if self.get_resource_amount("AdvancedSpecial"):
                keys_to_extract = ["Glass", "Paper"]
                advanced_trade_ratio = {}
                for key in keys_to_extract:
                    if key in trade_ratios:
                        advanced_trade_ratio[key] = trade_ratios[key]
                for i in range(self.get_resource_amount("AdvancedSpecial")):
                    if m_dict[max(advanced_trade_ratio, key=advanced_trade_ratio.get)]>0:
                        m_dict[max(advanced_trade_ratio, key=advanced_trade_ratio.get)] -= 1
            #sum up remaining resources and multiple m_dict by trade_calc_ratios.
            card_total_coin = card.get_coin_cost() + sum({k: m_dict.get(k, 0) * trade_ratios.get(k, 0) for k in set(m_dict)}.values())
            if card_total_coin:
                print(self.get_name() + " doesn't have the necessary resources to gain this card\n" +
                      self.get_name() +" is missing "
                      )
                print(m_dict)
                q = input("The smallest cost for the card is {0}. Do you want to purchase?\nYou have {1} coins".
                          format(card_total_coin, self.get_coin()))
                if q == "Yes": #change coin amount to replace
                    if self.get_coin() - card_total_coin >= 0: 
                        self.put_card(card, card_total_coin)
                    else:
                        print(self.get_name() + " doesn't have enough coins, please choose something else")
                else:
                    print(self.get_name() + " doesn't acquire {0}, please choose again".format(card.get_name()))
            else:
                self.put_card(card, card_total_coin)

# Card Class

In [3]:
 class Card: 
    #must define card type and name everything else is optional 
    #attack = number of attack positions army moves
    #vp = standalone vp points
    #coin = coins to gain
    #cost = dictionary of costs (including coin)
    #resource = resources to gain (not including coin)
    #card_type = resource/attack/vp/ect (good for counting what you have at end of game).
    def __init__(self, name='', attack=0, vp=0, coin=0, cost={"Coin": 0}, resource={}, card_type=''):
        self.name = name
        self.attack  = attack
        self.vp = vp
        self.coin = coin
        self.cost=cost
        self.resource=resource
        self.card_type=card_type
    def get_name(self):
        return self.name  
    def get_attack(self):
        return self.attack
    def get_vp(self):
        return self.vp
    def get_coin(self):
        return self.coin
    def get_cost(self):
        return self.cost
    def get_coin_cost(self):
        if "Coin" in self.get_cost():
            return self.get_cost()["Coin"]
        else:
            return 0
    def get_resource(self):
        return self.resource
    def get_card_type(self):
        return self.card_type

# Define Gameboard 

In [4]:
#Randomly select x cards from card_list
#

# Card Dictionary
## Use this space to define cards that are currently in play

In [5]:
# Need to set up so only have to define cards with non-zero values. 
# Dumb to make it so you have to write 0 over and over.
# cards = [Card('Guard Tower', 1, 0, 0, {"Coin":1}, {"Wood":0, "Brick":0, "Stone":0, "Glass":0, 
#                                           "Paper":0, "BaseSpecial":0, "AdvancedSpecial":0}, "Attack"), 
#          Card('Stone Pit', 0, 0, 0, {"Coin":0}, {"Wood":0, "Brick":0, "Stone":1, "Glass":0, "Paper":0}, "Resource")]
# # player_list=[Player("Alex"), Player("Jess")]
# Caravansery = Card(name='Caravansery', attack=0, vp=0, coin=0, cost={"Coin":2, "Glass":1, "Paper":1}, 
#                    resource={"BaseSpecial":1}, card_type="Treasure")


#Guard_Tower = Card(name='Guard Tower', attack=1, coin={"Coin":1}, card_type="Attack")
#Stone_Pit = Card(name='Stone Pit', coin={"Coin":1}, resource={"Stone":1}, card_type="Resource")
# player_list=[Player("Alex"), Player("Jess")]
Glassworks = Card(name = 'Glassworks', cost = {"Coin": 1}, resource={"Glass":1}, card_type="Resource")
Stables = Card(name = 'Stables', attack = 1, cost = {"Glass": 2, "Stone":1}, card_type="Attack")
Caravansery = Card(name='Caravansery', cost={"Coin":2, "Glass":1, "Paper":1}, 
                   resource={"AdvancedSpecial":1, "BaseSpecial":1}, card_type="Treasure")
#go back through and add checks for coins
p1 = Player("Alex")
p2 = Player("Jess")
#p1.add_card(Guard_Tower, p2)
#p1.add_card(Stone_Pit, p2)
#p1.add_card(Caravansery, p2)
p2.add_card(Glassworks, p1)
p1.add_card(Caravansery, p2)
p1.add_card(Stables, p2)


Jess acquired Glassworks
Jess spent 1 Coin(s)
Alex doesn't have the necessary resources to gain this card
Alex is missing 
{'Glass': 1, 'Paper': 1}


The smallest cost for the card is 7. Do you want to purchase?
You have 7 coins No


Alex doesn't acquire Caravansery, please choose again
Alex doesn't have the necessary resources to gain this card
Alex is missing 
{'Glass': 2, 'Stone': 1}


The smallest cost for the card is 8. Do you want to purchase?
You have 7 coins Yes


Alex doesn't have enough coins, please choose something else


# Test Area

4